# R3 Q8: Heterogeneity Definition

## Reviewer Question

**Referee #3, Question 8**: "You talk about heterogeneity, both patient and biological heterogeneity. In the literature one sometimes talks about disease heterogeneity, but it's not clear to me whether that's what you mean. Please clarify what you mean by heterogeneity."

## Why This Matters

Clarifying heterogeneity is essential for:
- Understanding what signatures capture
- Interpreting model outputs
- Communicating findings to clinical audiences

## Our Approach

We clarify heterogeneity through **pathway analysis** that demonstrates:

1. **Patient Heterogeneity**: Different individuals with same diagnosis have different signature profiles
2. **Biological Heterogeneity**: Same phenotype arises from different biological pathways
3. **Disease Heterogeneity**: Umbrella term capturing that clinical diagnoses are not single entities

---

## Key Findings

✅ **4 distinct pathways** to myocardial infarction identified  
✅ **44.8% of MI patients** follow "Hidden Risk" pathway (minimal pre-existing disease)  
✅ **Different pathways** show different signature patterns and genetic risk profiles

---


## Pathway Analysis: 4 Distinct Pathways to Myocardial Infarction

We identified 4 distinct biological pathways to MI using deviation-from-reference clustering (10-year lookback).


In [ ]:
import pandas as pd

# Pathway summary (from heterogeneity analysis)
pathways = pd.DataFrame({
    'Pathway': ['Pathway 0: Progressive Ischemia', 'Pathway 1: Hidden Risk', 
                'Pathway 2: Multimorbid Inflammatory', 'Pathway 3: Metabolic'],
    'Size': [1836, 11108, 4439, 7420],
    'Percent': [7.4, 44.8, 17.9, 29.9],
    'Age at MI': [70, 66, 72, 62],
    'Key Characteristics': [
        '86% have CAD (highest), chronic progressive disease',
        'Minimal pre-existing disease (8% CAD), low genetic risk',
        '35% arthropathy, 26% GI disease, inflammatory pathway',
        '32% diabetes (highest), metabolic syndrome, youngest onset'
    ]
})

print("="*80)
print("PATHWAYS TO MYOCARDIAL INFARCTION")
print("="*80)
display(pathways)


## Why Signatures vs. Diseases? The Aggregation Principle

**Analogy: ICD Codes → Phecodes → Signatures**

Just as **Phecodes aggregate multiple ICD-10 codes** into clinically meaningful disease categories, **signatures aggregate multiple diseases** into biologically meaningful pathways.

### ICD vs. PheCode Comparison

- **ICD-10 codes**: 1,133 distinct codes for cardiovascular diseases
- **Phecodes**: 53 aggregated categories
- **Reduction**: 21.4× fewer predictions needed (53 vs 1,133)
- **Average**: 28.7 ICD-10 codes per PheCode

This demonstrates that **aggregation reveals higher-level patterns** that individual codes miss.

### Signatures vs. Diseases (Same Principle)

- **Individual diseases**: 348 distinct disease codes
- **Signatures**: 20 disease signatures + 1 healthy reference
- **Reduction**: 16.6× fewer dimensions (21 vs 348)
- **Benefit**: Signatures capture shared biological pathways across diseases

**Key Insight**: Just as Phecodes reveal that "multiple ICD codes → one disease", signatures reveal that "multiple diseases → one biological pathway".

---

## Definitions of Heterogeneity

### 1. Patient Heterogeneity
- **Definition**: Different individuals with the same disease diagnosis have different signature profiles
- **Example**: Two patients with CAD may have different metabolic vs. inflammatory signatures
- **Measurement**: Average pairwise distance in signature space
- **Finding**: Average within-disease signature distance = 3.87 (substantial heterogeneity)

### 2. Biological Heterogeneity
- **Definition**: Same clinical phenotype can arise from different biological pathways
- **Example**: CAD can be driven by metabolic dysfunction, inflammation, or genetic factors
- **Measurement**: Signature diversity within disease groups
- **Finding**: 4 distinct pathways to MI demonstrate biological heterogeneity

### 3. Disease Heterogeneity (What We Mean)
- **Definition**: Umbrella term encompassing both patient and biological heterogeneity
- **Observation**: "CAD" is not a single entity but a collection of related conditions
- **Model Capture**: Individual-specific signature loadings capture this heterogeneity


In [ ]:
# ============================================================================
# ICD vs PHE CODE COMPARISON
# ============================================================================
"""
Shows the aggregation principle: ICD codes → Phecodes → Signatures

This demonstrates why signatures (aggregated patterns) are more informative
than individual disease codes, similar to how Phecodes are more informative
than individual ICD-10 codes.
"""

import pandas as pd
from pathlib import Path

# Load ICD to PheCode mapping
mapping_path = Path('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/icd2phecode_mergedwithdetailedphecode_info.csv')

if mapping_path.exists():
    print("="*80)
    print("ICD vs PHE CODE COMPARISON")
    print("="*80)
    
    mapping_df = pd.read_csv(mapping_path)
    
    # Count unique ICD-10 codes and Phecodes
    # Assuming the mapping has columns like 'ICD10' and 'phecode'
    # Adjust column names based on actual structure
    print(f"\nLoaded mapping file: {mapping_path.name}")
    print(f"Total rows: {len(mapping_df):,}")
    print(f"Columns: {mapping_df.columns.tolist()}")
    
    # Try to identify ICD and PheCode columns
    icd_cols = [c for c in mapping_df.columns if 'icd' in c.lower() and '10' in c.lower()]
    phe_cols = [c for c in mapping_df.columns if 'phecode' in c.lower() or 'phe' in c.lower()]
    
    if icd_cols and phe_cols:
        icd_col = icd_cols[0]
        phe_col = phe_cols[0]
        
        n_unique_icd = mapping_df[icd_col].nunique()
        n_unique_phe = mapping_df[phe_col].nunique()
        
        print(f"\n📊 Aggregation Statistics:")
        print(f"   Unique ICD-10 codes: {n_unique_icd:,}")
        print(f"   Unique Phecodes: {n_unique_phe:,}")
        print(f"   Reduction factor: {n_unique_icd / n_unique_phe:.1f}×")
        print(f"   Average ICD codes per PheCode: {n_unique_icd / n_unique_phe:.1f}")
        
        # Show example: how many ICD codes map to common Phecodes
        phe_counts = mapping_df.groupby(phe_col)[icd_col].nunique().sort_values(ascending=False)
        print(f"\n📋 Examples (top 10 Phecodes by ICD count):")
        print(f"{'PheCode':<15} {'N_ICD_Codes':<15} {'ICD_Codes_per_PheCode':<25}")
        print("-" * 60)
        for phe, n_icd in phe_counts.head(10).items():
            print(f"{str(phe):<15} {n_icd:<15} {n_icd:.1f}")
        
        print(f"\n💡 Key Insight:")
        print(f"   Just as Phecodes aggregate {n_unique_icd:,} ICD codes into {n_unique_phe:,} categories,")
        print(f"   Signatures aggregate 348 diseases into 21 biological pathways.")
        print(f"   This reveals higher-level patterns that individual codes/diseases miss.")
    else:
        print(f"\n⚠️  Could not identify ICD/PheCode columns automatically")
        print(f"   ICD-like columns: {icd_cols}")
        print(f"   PheCode-like columns: {phe_cols}")
        print(f"\n   Showing first few rows:")
        print(mapping_df.head())
else:
    print("="*80)
    print("ICD vs PHE CODE COMPARISON")
    print("="*80)
    print(f"\n⚠️  Mapping file not found: {mapping_path}")
    print(f"\n📊 Summary (from analysis in delphicomp.ipynb):")
    print(f"   - 1,133 ICD-10 codes → 53 Phecodes (cardiovascular)")
    print(f"   - Reduction: 21.4× fewer predictions needed")
    print(f"   - Average: 28.7 ICD codes per PheCode")
    print(f"\n💡 This demonstrates the aggregation principle:")
    print(f"   ICD codes → Phecodes → Signatures")
    print(f"   Individual codes → Disease categories → Biological pathways")


## Summary & Response Text

### Key Findings

1. **Why Signatures vs. Diseases?**: Just as Phecodes aggregate 1,133 ICD-10 codes into 53 categories (21.4× reduction), signatures aggregate 348 diseases into 21 biological pathways (16.6× reduction). This aggregation reveals higher-level patterns that individual codes/diseases miss.

2. **Patient Heterogeneity**: Different individuals with same diagnosis show different signature profiles (average distance = 3.87)

3. **Biological Heterogeneity**: Same phenotype (MI) arises from 4 distinct pathways

4. **Disease Heterogeneity**: Clinical diagnoses are not single entities—our model captures this through individual-specific signatures

### Response to Reviewer

> "We clarify our use of 'heterogeneity' as follows: (1) **Patient Heterogeneity**: Different individuals with the same disease diagnosis have different signature profiles (measured by average pairwise distance in signature space = 3.87). For example, two patients with CAD may have different metabolic vs. inflammatory signatures. (2) **Biological Heterogeneity**: Same clinical phenotype can arise from different biological pathways. For example, CAD can be driven by metabolic dysfunction, inflammation, or genetic factors. We demonstrate this through pathway analysis identifying 4 distinct pathways to myocardial infarction (progressive ischemia, hidden risk, inflammatory, metabolic). (3) **Disease Heterogeneity**: Umbrella term encompassing both patient and biological heterogeneity—the observation that 'CAD' is not a single entity but a collection of related conditions. Our model captures this through individual-specific signature loadings. We now consistently use these terms throughout the manuscript and define them in Methods.
>
> **Why use signatures instead of diseases?** The same aggregation principle that makes Phecodes more informative than individual ICD-10 codes applies to signatures versus diseases. Just as Phecodes aggregate 1,133 ICD-10 codes into 53 clinically meaningful categories (21.4× reduction), signatures aggregate 348 diseases into 21 biologically meaningful pathways (16.6× reduction). This reveals shared biological mechanisms across diseases that individual disease codes cannot capture."

### References

- Pathway analysis: `heterogeneity_analysis_summary.ipynb`
- Pathway discovery: `run_complete_pathway_analysis_deviation_only.py`
- ICD vs PHE comparison: `delphicomp.ipynb`, `compare_icd10_aggregation.py`
